In [1]:
import numpy as np
import pandas as pd
import lightgbm as lgb
import datetime
import math
import gc
import pickle
from sklearn import metrics

In [2]:
data_path = '../data/'
train = pickle.load( open(data_path + 'train_feature_myadding1_reduceDim', 'rb'))
print 'loading done'

loading done


In [3]:
data_path = '../data/'
test = pickle.load( open(data_path + 'test_feature_myadding1_reduceDim', 'rb'))
print 'loading done'

loading done


In [7]:
cols = ['artist_composer', 'artist_composer_lyricist', 'song_lang_boolean', 'smaller_song', 'count_song_played', \
        'count_artist_played', 'song_country']
# 29. artist_composer (0.000000)
# 30. artist_composer_lyricist (0.000000)
# 31. song_lang_boolean (0.000000)
# 32. smaller_song (0.000000)
# 33. count_song_played (0.000000)
# 34. count_artist_played (0.000000)
# 35. song_country (0.000000)
#train = train.drop(cols, axis = 1)
test = test.drop(cols, axis = 1)

In [5]:
print train.dtypes

msno                  category
song_id                  int64
source_system_tab     category
source_screen_name    category
source_type           category
target                   uint8
song_length             uint32
genre_ids             category
artist_name           category
composer              category
lyricist              category
language              category
city                  category
bd                       uint8
gender                category
registered_via        category
expiration_date          int64
registration_year        int64
registration_month       int64
registration_date        int64
expiration_year          int64
expiration_month         int64
membership_days          int64
genre_ids_count           int8
lyricists_count           int8
composer_count            int8
is_featured               int8
artist_count              int8
song_year_y            float64
dtype: object


In [8]:
# pickle.dump(train, open(data_path + 'train_feature_myadding', 'wb'))
# pickle.dump(test, open(data_path + 'test_feature_myadding', 'wb'))

In [8]:
print ("Train test and validation sets")
for col in train.columns:
    if train[col].dtype == object:
        train[col] = train[col].astype('category')
        test[col] = test[col].astype('category')


X_train = train.drop(['target'], axis=1)
y_train = train['target'].values


X_test = test.drop(['id'], axis=1)
ids = test['id'].values

length = int(len(X_train)*0.8)
X_trainSet = X_train[:length]
y_trainSet = y_train[:length]
X_valSet = X_train[length:]
y_valSet =  y_train[length:]
# del train, test; gc.collect();

# X_trainSet = X_train
# y_trainSet = y_train
# X_valSet = X_train
# y_valSet =  y_train

d_train_final = lgb.Dataset(X_trainSet, y_trainSet)
watchlist_final = lgb.Dataset(X_valSet, y_valSet)
train_final = lgb.Dataset(X_train, y_train)
print('Processed data...')

Train test and validation sets
Processed data...


In [9]:
def lgb_model(max_depth = 16, tree = 'gbdt', fold = 5, max_bin = 50, bagging_fraction = 0.95 ):
    n_round = 200 + (15 - max_depth) * 25
    params = {
        'objective': 'binary',
        'metric': 'binary_logloss',
        'boosting': tree,
        'learning_rate': 0.6,
        'verbose': 0,
        'num_leaves': 108,
        'bagging_fraction': bagging_fraction,
        'bagging_freq': 1,
        'bagging_seed': 1,
        'feature_fraction': 0.9,
        'feature_fraction_seed': 1,
        'max_bin': max_bin,
        'max_depth': max_depth,
        'num_rounds': 200,
        'metric' : 'auc'
    }
    frac = 1 - (1.0 / fold)
    evals_result = {}
    model_f = lgb.train(params, train_set=d_train_final,  valid_sets=watchlist_final, verbose_eval=200,\
                              evals_result=evals_result)
    print 'done'
    p_test = model_f.predict(X_test)
    print  'mean=', evals_result['valid_0']['auc'][-1]
    return [p_test, evals_result['valid_0']['auc'][-1]]

In [10]:
def lgb_model_final(max_depth = 16, tree = 'gbdt', fold = 5, max_bin = 50, bagging_fraction = 0.95 ):
    n_round = 200 + (max_depth - 15) * 5
    params = {
        'objective': 'binary',
        'metric': 'binary_logloss',
        'boosting': tree,
        'learning_rate': 0.6,
        'verbose': 0,
        'num_leaves': 108,
        'bagging_fraction': bagging_fraction,
        'bagging_freq': 1,
        'bagging_seed': 1,
        'feature_fraction': 0.9,
        'feature_fraction_seed': 1,
        'max_bin': max_bin,
        'max_depth': max_depth,
        'num_rounds': 200,
        'metric' : 'auc'
    }
    evals_result = {}
    model_f = lgb.train(params, train_set=train_final,  valid_sets=train_final, verbose_eval=200,\
                              evals_result=evals_result)
    print 'done'
    p_test = model_f.predict(X_test)
    print  'mean=', evals_result['training']['auc'][-1]
    return [p_test, evals_result['training']['auc'][-1]]

In [12]:
weights, p_tests = [], []
max_bin = 50
for tree in ['dart', 'gbdt']:
    print tree
    for max_depth in range(15,23,1):
        print 'max_depth...', max_depth
        p_test, res = lgb_model(tree = tree, max_depth = max_depth)
        weights += res,
        p_tests += p_test,

dart
max_depth... 15
[200]	valid_0's auc: 0.669546
done
mean= 0.669546420663
max_depth... 16
[200]	valid_0's auc: 0.668956
done
mean= 0.668955643671
max_depth... 17
[200]	valid_0's auc: 0.670751
done
mean= 0.670751241843
max_depth... 18
[200]	valid_0's auc: 0.674192
done
mean= 0.674192168228
max_depth... 19
[200]	valid_0's auc: 0.672508
done
mean= 0.672508307597
max_depth... 20
[200]	valid_0's auc: 0.672112
done
mean= 0.672111574233
max_depth... 21
[200]	valid_0's auc: 0.673609
done
mean= 0.673608651981
max_depth... 22
[200]	valid_0's auc: 0.674405
done
mean= 0.674405060155
gbdt
max_depth... 15
[200]	valid_0's auc: 0.666838
done
mean= 0.666837814843
max_depth... 16
[200]	valid_0's auc: 0.666437
done
mean= 0.666437252796
max_depth... 17
[200]	valid_0's auc: 0.668329
done
mean= 0.668329181874
max_depth... 18
[200]	valid_0's auc: 0.666712
done
mean= 0.666711987665
max_depth... 19
[200]	valid_0's auc: 0.667165
done
mean= 0.667164510717
max_depth... 20
[200]	valid_0's auc: 0.668275
done
mea

In [36]:
weights, p_tests = [], []
max_bin = 50
for tree in ['dart']:
    print tree
    for max_depth in range(28,31,1):
        print 'max_depth...', max_depth
        p_test, res= lgb_model_final(tree = tree, max_depth = max_depth)
        weights += res,
        p_tests += p_test,

dart
max_depth... 28
[200]	training's auc: 0.837109
done
mean= 0.837109407473
max_depth... 29
[200]	training's auc: 0.837426
done
mean= 0.837426078896
max_depth... 30
[200]	training's auc: 0.837214
done
mean= 0.837213799781


In [37]:
weights1 = [(x * 10) ** 2 for x in weights]

In [38]:
print len(weights1)

3


In [39]:
weights = weights1

In [ ]:
# pickle.dump(p_tests, open(data_path + 'p_tests', 'wb'))
# pickle.dump(weights1, open(data_path + 'weights', 'wb'))

In [40]:
result = (np.array(p_tests).T * np.array(weights)).T
print('Done making predictions')

Done making predictions


In [41]:
print (np.shape(result))
total = np.sum(weights)
mat = np.sum(np.matrix(result), axis = 0)
p_test = [x / total for x in (mat.tolist()[0])]
print (np.shape(p_test))
for x in p_test[:3]:
    print (x)

(3, 2556790)
(2556790,)
0.391422638034
0.372698401874
0.17617924762


In [43]:
print ('Saving predictions Model model of gbdt')
subm = pd.DataFrame()
subm['id'] = ids
subm['target'] = p_test
subm.to_csv(data_path + '../submissions/submission_lgbm_27_30.csv.gz', compression = 'gzip', index=False, float_format = '%.5f')

print('Done!')

Saving predictions Model model of gbdt
Done!


In [ ]:
print data_path

In [ ]:
# dart
# max_depth... 22
# [200]	valid_0's auc: 0.683127
# done
# mean= 0.683126554982
# max_depth... 23
# [200]	valid_0's auc: 0.684025
# done
# mean= 0.684025200776
# max_depth... 24
# [200]	valid_0's auc: 0.683424
# done
# mean= 0.683423870829
# max_depth... 25
# [200]	valid_0's auc: 0.683243
# done
# mean= 0.683242848975
# max_depth... 26
# [200]	valid_0's auc: 0.684003
# done
# mean= 0.684003282544
# max_depth... 27
# [200]	valid_0's auc: 0.683639
# done
# mean= 0.683639398343
# max_depth... 28
# [200]	valid_0's auc: 0.683466
# done
# mean= 0.68346612941
# max_depth... 29
# [200]	valid_0's auc: 0.683804
# done
# mean= 0.683804281482
# max_depth... 30
# [200]	valid_0's auc: 0.684491
# done
# mean= 0.684490583375
# gbdt
# max_depth... 22
# [200]	valid_0's auc: 0.67788
# done
# mean= 0.677880199916
# max_depth... 23
# [200]	valid_0's auc: 0.676867
# done
# mean= 0.676866752141
# max_depth... 24
# [200]	valid_0's auc: 0.677996
# done
# mean= 0.677996261719
# max_depth... 25
# [200]	valid_0's auc: 0.677539
# done
# mean= 0.677538786505
# max_depth... 26
# [200]	valid_0's auc: 0.678415
# done
# mean= 0.678414907137
# max_depth... 27
# [200]	valid_0's auc: 0.678192
# done
# mean= 0.678191940942
# max_depth... 28
# [200]	valid_0's auc: 0.677959
# done
# mean= 0.677958934205
# max_depth... 29
# [200]	valid_0's auc: 0.67775
# done
# mean= 0.677749929258
# max_depth... 30
# [200]	valid_0's auc: 0.678718
# done
# mean= 0.678718033285